In [60]:
import numpy as np
from scipy.optimize import minimize_scalar
import numpy.linalg as ln
import random
import scipy.optimize

In [61]:
def rosenbrock(x):
    
    global calls
    calls += 1
    
    n = len(x)
    res = 0
    for i in range(n - 1):
        res += 100.0 * (x[i + 1] - x[i]**2)**2 + (1 - x[i])**2
    return res

In [62]:

calls = 0


epsilon = 1e-7
delta = 1e-3
n_sum = int (input("enter N:"))

#x0 = np.array([float(val) for val in input("Введіть початкову точку x0 через пробіл: ").split()])
#x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0])
x0 = np.full(n_sum, 0)
max_iterations = int(input("Введіть максимальну кількість ітерацій (K < 1000): "))

In [63]:
def rosenbrock_gradient(x):
    
    global calls
    calls += len(x)
    
    if len(x) < 2:
        raise ValueError("Для обчислення градієнта x повинен мати розмірність принаймні 2.")
    
    n = len(x)
    gradient = np.zeros_like(x)
    gradient[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    gradient[1:-1] = 200 * (x[1:-1] - x[:-2]**2) - 400 * x[1:-1] * (x[2:] - x[1:-1]**2) - 2 * (1 - x[1:-1])
    gradient[-1] = 200 * (x[-1] - x[-2]**2)
    return gradient

In [64]:
def golden_section_search(f, a, b, tol=1e-5):
    gr = (np.sqrt(5) + 1) / 2 - 1
    c = b - gr * (b - a)
    d = a + gr * (b - a)

    while abs(c - d) > tol:
        if f(c) < f(d):
            b = d
        else:
            a = c

        c = b - gr * (b - a)
        d = a + gr * (b - a)

    return (b + a) / 2

In [65]:
def bfgs_method(x0, elipson, delta, K):
    x = x0
    k = 0
    N = len(x0)
    I = np.eye(N, dtype=int)
    Hk = I
   #################
    gradient = rosenbrock_gradient(x)
    

    while ln.norm(gradient) > elipson and k < K:
        f_x = rosenbrock(x)
    
        pk = -np.dot(Hk, gradient)

        objective_function = lambda alpha: rosenbrock(x + alpha * pk)
        alpha_k = golden_section_search(objective_function, 0, 1, tol=delta)

        xkp1 = x + alpha_k * pk
        sk = xkp1 - x
        x = xkp1

        fderxp1 = rosenbrock_gradient(xkp1)
        yk = fderxp1 - gradient
        gradient = fderxp1


        k += 1
        
        ro = 1.0 / (np.dot(yk, sk))
        A1 = I - ro * sk[:, np.newaxis] * yk[np.newaxis, :]
        A2 = I - ro * yk[:, np.newaxis] * sk[np.newaxis, :]
        Hk = np.dot(A1, np.dot(Hk, A2)) + (ro * sk[:, np.newaxis] * sk[np.newaxis, :])
        
    return x, f_x, np.linalg.norm(pk), k





In [66]:
result = bfgs_method(x0, epsilon, delta, max_iterations)

print("Optimal point:", result[0])
print("Optimal value:", result[1])
print("Norm of gradient:", result[2])
print("Number of iterations:", result[3])
print("Number of gradient calls: ", calls)

Optimal point: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Optimal value: 7.745732094279547e-17
Norm of gradient: 9.418821712534409e-10
Number of iterations: 124
Number of gradient calls:  15600
